In [1]:
import json
#import rpy2.robjects as robjects
import  graph_construction as function
#import enrichment_image as enrichment
import os
import rpy2
import logging
import rpy2.robjects as robjects
from rpy2.rinterface_lib.callbacks import logger as rpy2_logger
rpy2_logger.setLevel(logging.ERROR)

#CARICAMENTO FILE CONFIGURAZIONE
with open("config.json") as f:
    data=json.load(f)

#Definizione pathway
path_data_mutational=data["Paths"]["data_mutational"]
path_clinical_sample=data["Paths"]["data_clinical_sample"]
path_clinical_patient=data["Paths"]["data_clinical_patient"]
column_mutation_name=data["Mutation"]["column_mutation_name"]
gene_interest=data["Mutation"]["gene_of_interest"]
path_save=data["Paths"]["output_folder"]
column_gene=data["Mutation"]["column_gene_name"]
column_hgvsp=data["Mutation"]["column_hgvsp_short"]
#column_hgvsc=data["Mutation"]["column_hgvsc"]
column_variant_classification=data["Mutation"]["column_variant_classification"]
column_chromosome=data["Mutation"]["column_chromosome"]
column_start=data["Mutation"]["column_start"]
column_end=data["Mutation"]["column_end"]
sample_name=data["Clinical_data"]["column_sample_name"]
patient_name=data["Clinical_data"]["column_patient_name"]
vaf=data["Mutation"]["vaf"]
vaf_score=data["Mutation"]["vaf_score"]
column_vaf=data["Mutation"]["vaf_column"]
adjusted=data["Enrichment"]["adjusted"]
threshold=data["Enrichment"]["threshold"]
go=data["Enrichment"]["go"]
kegg=data["Enrichment"]["kegg"]
wiki=data["Enrichment"]["wiki"]
reactome=data["Enrichment"]["reactome"]

 #creazione cartella di output
#if not os.path.exists(path_save):
    #os.makedirs(path_save)


In [4]:
k_fold_method=True
k_fold=4
k=3

In [5]:
if not os.path.exists(f"{path_save}_cross_validation_{k}"):
    os.makedirs(f"{path_save}_cross_validation_{k}")
path_save=f"{path_save}_cross_validation_{k}"

In [ ]:
#caricamento dei dataframes
data_mutational,data_clinical_sample,data_clinical_patient=function.read_file(path_data_mutational,path_clinical_sample,path_clinical_patient)
#creazione delle mappe pazienti e varianti + creazione del grafo
if column_mutation_name=="":
    #print("ciao")
    data_mutational=function.adding_category_mutation(data_mutational,column_gene,column_hgvsp,column_variant_classification,column_chromosome,column_start,column_end)


if vaf and column_vaf!="":
    data_mutational=data_mutational[data_mutational[column_vaf]>=vaf_score]
elif vaf and column_vaf=="":
    data_mutational['t_AF']=data_mutational.apply(function.calculated_vaf, axis=1)
    print(len(data_mutational))
    data_mutational=data_mutational[data_mutational['t_AF']>=vaf_score]
    print(len(data_mutational))

map_patients,map_variants,map_consequence=function.create_maps(data_mutational,data,column_mutation_name)

graph=function.graph_creation(map_patients,map_variants)

In [ ]:
if k_fold_method:
    all_samples=data_mutational["Tumor_Sample_Barcode"].unique()
    all_samples_filtred=[]
    block_division=len(all_samples)//k_fold
    for i,e in enumerate(all_samples):
        if i >=(k*block_division) and i <(k*block_division)+block_division:
            continue
        all_samples_filtred.append(e)
    data_mutational=data_mutational[data_mutational["Tumor_Sample_Barcode"].isin(all_samples_filtred)]
    